In [1]:
import json
import os

from dotenv import load_dotenv
from typing import Optional, List, Mapping, Any

In [2]:
load_dotenv(".env")

True

In [3]:
from openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core import Settings

In [4]:
class OurLLM(CustomLLM):
    context_window: int = 128e3
    num_output: int = 8192
    model_name: str = "nebius"
    dummy_response: str = "Dummy response"

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        client = OpenAI(
            base_url="https://api.studio.nebius.ai/v1/",
            api_key=os.environ.get("NEBIUS_API_KEY"),
        )

        completion = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-405B-Instruct",
            messages=[
            {
                "role": "system",
                "content": prompt
            }
            ],
            temperature=0,
            max_tokens=8192,
            top_p=0.9
        )

        return CompletionResponse(
            text=completion.choices[0].message.content
        )
    
    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        response = ""
        for token in self.dummy_response:
            response += token
            yield CompletionResponse(text=response, delta=token)

/Users/stas_chi/Library/Caches/pypoetry/virtualenvs/izba_ai-UUHFOrDR-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in OurLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

def divide(a: float, b: float) -> float:
    return a/b

divide_tool = FunctionTool.from_defaults(fn=divide)

In [6]:
import requests

In [14]:
URL = "https://api.tavily.com/search"

# Define the parameters for the API request
payload = {
    "api_key": os.environ.get("TAVILY_API_KEY"),
    "query": "What is the latest share price of LSEG?",
    "search_depth": "basic",
    "include_answer": True,
    "include_images": False,
    "include_image_descriptions": False,
    "include_raw_content": False,
    "max_results": 5,
    "include_domains": [],
    "exclude_domains": []
}

# Set the appropriate headers (if needed, such as for content type)
headers = {
    "Content-Type": "application/json"
}

In [15]:
response = requests.post(URL,headers=headers,data=json.dumps(payload))

In [17]:
response.json()["answer"]

'The latest share price of London Stock Exchange Group plc (LSEG) is currently trading at a record high, having surged by over 12% this year, outperforming the FTSE 100 and FTSE 250 indices.'

## Initialise Agent

In [6]:
llm = OurLLM()

In [7]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, divide_tool], llm=llm, verbose=True)

In [8]:
response = agent.chat(
    "Solve the following equation: 10 = 5x"
)

/Users/stas_chi/Library/Caches/pypoetry/virtualenvs/izba_ai-UUHFOrDR-py3.11/lib/python3.11/site-packages/llama_index/core/schema.py:79: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `float` with value `128000.0` - serialized value may not be as expected
  data = handler(self)


> Running step 1243b793-cda7-4658-8595-d35bcd00fb91. Step input: Solve the following equation: 10 = 5x
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: divide
Action Input: {'a': 10, 'b': 5}
Observation: 2.0
> Running step 78ebb35c-e880-4cda-a76a-45bb7b06c79b. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The value of x is 2.0.
